https://www.geeksforgeeks.org/webcam-motion-detector-python/

In [1]:
import numpy as np
import cv2

import time

In [4]:
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG(backgroundRatio=0.8)

# Assigning our static_back to None 
static_back = None
  
# Capturing video 
video = cv2.VideoCapture('/home/roman/projects/free_parking_space/data/our_parking_car_try_parking2_day.mp4')

# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# writer = cv2.VideoWriter('/home/roman/projects/free_parking_space/data/test.mp4',fourcc, 20,(1280,1024))


# MIN_SIZE_OBJECT = 2000
MIN_SIZE_OBJECT = 6000
TRESHOLD = 30
BLUR_FILTER_SIZE = (3, 3)

while video.isOpened():
    check, frame = video.read() 
    if not check:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    gray = cv2.GaussianBlur(gray, BLUR_FILTER_SIZE, 0)
    
    fgmask = fgbg.apply(gray)

    # for first iteration 
    if static_back is None: 
        static_back = fgmask 
        continue

    # Difference between background and curent frame   
    diff_frame = cv2.absdiff(static_back, fgmask) 

    # If change in between static background and 
    # current frame is greater than TRESHOLD it will show white color(255) 
    thresh_frame = cv2.threshold(diff_frame, TRESHOLD, 255,cv2.THRESH_BINARY)[1] 
    thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2) 

    # Finding contour of moving object 
    cnts, _ = cv2.findContours(thresh_frame.copy(),  
                               cv2.RETR_EXTERNAL, 
                               cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in cnts: 
        if cv2.contourArea(contour) > MIN_SIZE_OBJECT: 

            (x, y, w, h) = cv2.boundingRect(contour) 
            # making green rectangle arround the moving object 
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
            
            x_center = int(x+0.5*w)
            y_center = int(y+0.5*h)
            cv2.circle(frame,(x_center,y_center), 1, (0,0,255), 2)



    cv2.imshow("Difference Frame", diff_frame) 
#     cv2.imshow("Color Frame", frame)
#     writer.write(frame)

    if cv2.waitKey(33) == 27:
            break
        

video.release()
# writer.release()
cv2.destroyAllWindows() 